# Linear threshold LT model

## Extraction of testing methods

In [1]:
from matplotlib import pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

shapes = ['o', 'v', '^', '<', '>', 's', 'p', '*', 'h', '+']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'darkgreen', 'chocolate']

In [2]:
RCIs = dict()
DCs = dict()
BCs = dict()
CCs = dict()
PGs = dict()
ERs = dict()
AFOs = dict()
AFPs = dict()

import os

from experiment.functions import func_get_lab_graph
from experiment.lab2._config_ import SoftwareConfigs1, LabSpace

Graphs = dict()
Labels = dict()
Types = dict()


for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        modelDir = os.path.join(LabSpace, softwareName)
        labDir = os.path.join(modelDir, '.lab2')

        if not os.path.isdir(labDir):
            os.makedirs(labDir, exist_ok=True)

        G, labelDict, _, typeDict = func_get_lab_graph(modelDir, labDir, softwareName)
        Graphs[softwareName] = G
        Labels[softwareName] = labelDict
        Types[softwareName] = typeDict
        
        RCIs[softwareName] = dict(G.nodes.data('RCI'))
        DCs[softwareName] = dict(G.nodes.data('dc'))
        BCs[softwareName] = dict(G.nodes.data('bc'))
        CCs[softwareName] = dict(G.nodes.data('cc'))
        PGs[softwareName] = dict(G.nodes.data('pg'))
        ERs[softwareName] = dict(G.nodes.data('er'))
        AFOs[softwareName] = dict(G.nodes.data('AFO'))
        AFPs[softwareName] = dict(G.nodes.data('AFP'))

print('Extract data for risk diffusion experiments')

## LT Linear Threshold Model

### model parameter

In [4]:
from experiment.functions import func_get_edge_influence

Steps = 100

SimThreshold = 30

SimCount = 10

TestMethods = [
    ('DRI', RCIs),
    ('DC', DCs),
    ('BC', BCs),
    ('CC', CCs),
    ('PageRank', PGs),
    ('ElementRank', ERs),
]

Thresholds = dict()
Influences = dict()

for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName] 
        Influences[softwareName] = func_get_edge_influence(G, 'w-in-degree')
        Thresholds[softwareName] = {node: 0.5 for node in G.nodes}
        pass

LTSimDir = os.path.join(LabSpace, '.sim.LT')
if not os.path.isdir(LTSimDir):
    os.mkdir(LTSimDir)

## Linear threshold LT model simulation

### LT model parameter settings

In [5]:
Thresholds = dict()
Influences = dict()

for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName] 
        Influences[softwareName] = None
        Thresholds[softwareName] = {node: 0.5 for node in G.nodes}
        pass
    
SimDir = os.path.join(LabSpace, '.sim.LT')
if not os.path.isdir(SimDir):
    os.mkdir(SimDir)

### Perform simulation

In [6]:
import pandas as pd
import copy
from pandas import DataFrame
from experiment.algorithms.propagation_models.linear_threshold import linear_threshold
from experiment.functions import func_SortDictByValue

simResults = dict()
for softwareName, (status, language, src_dir) in SoftwareConfigs1.items():
    if status:
        G = Graphs[softwareName] 
        labels = Labels[softwareName]
        types = Types[softwareName]
        N = G.number_of_nodes()
        
        if isinstance(SimThreshold, float) and 0 <= SimThreshold <= 1:
            seedsCount = int(G.number_of_nodes() * SimThreshold)
        elif isinstance(SimThreshold, int) and 0 < SimThreshold < N:
            seedsCount = SimThreshold
        else:
            seedsCount = int(N * 0.15)
        
        print(f'{softwareName} starts simulating...')
        
        methodColumns = [methodName for methodName, _ in TestMethods]
        itemColumns = ['key', 'val', 'label', 'type']
        finalDf = DataFrame(
           columns = pd.MultiIndex.from_product(
               [methodColumns, itemColumns]
           )
        )
        
        seeds = set()
        MethodSeeds = dict()
        for methodName, methods in TestMethods:
            method = methods[softwareName]
            SortedMethod = func_SortDictByValue(method)
            SortedMethodIds = list(SortedMethod.keys())
            methodSeeds = SortedMethodIds[:seedsCount]
            MethodSeeds[methodName] = methodSeeds
            seeds.update(methodSeeds)
        
        print(f'\tStart simulating {len (seeds)} nodes')
        influences = Influences[softwareName]
        thresholds = Thresholds[softwareName]
        simVals = dict()
        for seed in seeds:
            acNodes, acNodesCounts, newAcNodes = linear_threshold(G, seeds=[seed], influences=influences, thresholds=thresholds)
            simVals[seed] = acNodesCounts[-1]
            pass
        
        simResults[softwareName] = dict()
        for methodName, methodSeeds in MethodSeeds.items():
            seedLabs = [labels[n] for n in methodSeeds]
            seedTypes = [types[n] for n in methodSeeds]
            seedVals = [simVals[n] for n in methodSeeds]
            
            totalVal = sum(seedVals)
            simResults[softwareName][methodName] = totalVal
            tSeeds = copy.deepcopy(methodSeeds)
            tSeeds.append(-1)
            seedVals.append(totalVal)
            seedLabs.append('total')
            seedTypes.append(methodName)
            
            finalDf[methodName, 'key'] = tSeeds
            finalDf[methodName, 'val'] = seedVals
            finalDf[methodName, 'label'] = seedLabs
            finalDf[methodName, 'type'] = seedTypes
        
        finalDf.to_csv(os.path.join(SimDir, f'{softwareName}.csv'))